1. 문서의 내용을 읽는다.
2. 문서를 쪼갠다.
- 토큰수 초과로 답변을 생성하지 못할 수 있고
- 문서가 길면 (인풋이 길면) 답변 생성이 오래거림
3. 임베딩 -> 벡터 데이터베이스에 저장
4. 질문이 있을때, 벡터 데이터베이스에 유사도 검색
5. 유사도 검색으로 가져온 문서를 LLM에 질문과 같이 전달

# 1. 문서의 내용을 읽는다.

In [ ]:
%pip install python-docx

In [ ]:
from docx import Document

document = Document('./tax.docx')
print(f'document == {dir(document)}')
full_text = ''
for index, para in enumerate(document.paragraphs):
    print(f'para == {para}')
    full_text += f'{para.text}\n'

print(full_text)



# 2. 문서를 쪼갠다

In [ ]:
%pip install tiktoken

In [11]:
import tiktoken

# encoder = tiktoken.encoding_for_model('gpt-4o')
# encoding = encoder.encode(full_text) # 토큰으로 변환 하지만 길이가 15만 토큰 이상이다.
# decoded = encoder.decode(encoding) # 그래서 쪼깬 문서단위로 가지고 0

def split_text(full_text, chunk_size):
  encoder = tiktoken.encoding_for_model('gpt-4o')
  total_encoding = encoder.encode(full_text)
  total_token_count = len(total_encoding)
  text_list = []

  for i in range(0, total_token_count, chunk_size):
    chunk = total_encoding[i:i+chunk_size]
    chunk_text = encoder.decode(chunk)
    text_list.append(chunk_text)

  return text_list

In [22]:
chunk_list = split_text(full_text, 1500)

# 3. 문서 임베딩

In [ ]:
%pip install chromadb

In [14]:
import chromadb

chroma_client = chromadb.Client()

In [15]:
collection_name = 'tax_collection'
tax_collection = chroma_client.get_or_create_collection(collection_name)

In [20]:
import os

from dotenv import load_dotenv
from chromadb.utils.embedding_functions import OpenAIEmbeddingFunction

load_dotenv()
open_api_key = os.getenv('OPENAI_API_KEY')

openai_embedding_function = OpenAIEmbeddingFunction(
  api_key=open_api_key,
  model_name='text-embedding-3-large',
)

In [21]:
tax_collection = chroma_client.get_or_create_collection(collection_name, embedding_function=openai_embedding_function)

In [23]:
id_list = []
for index in range(len(chunk_list)):
  id_list.append(f'{index}')

tax_collection.add(
  documents=chunk_list,
  ids=id_list
)

# 4. 유사도 검색

In [27]:
query = '연봉 1억인 직장인의 소득세는 얼마인가요?'

retrieved_docs = tax_collection.query(query_texts=[query], n_results=3)

# 5. LLM 질의

In [ ]:
%pip install openai

In [28]:
from openai import OpenAI
client = OpenAI()

response = client.chat.completions.create(
  model='gpt-4o',
  messages=[
    {'role': 'system', 'content': f"당신은 한국의 소득세 전문가 입니다. 아래 내용을 참고해서 답변해주세요 {retrieved_docs['documents'][0]}"},
    {'role': 'user', 'content': query}
  ]
)
response.choices[0].message.content

'연봉 1억 원인 직장인의 소득세를 계산하기 위해서는 몇 가지 추가 정보가 필요합니다. 우선 기본적인 소득세 계산을 위해서는 과세표준, 소득공제, 세액공제 등을 고려해야 합니다. 다만, 여기서는 일반적인 한국 소득세율과 누진세율을 근거로 대략적인 소득세를 계산해드리겠습니다.\n\n1. **과세표준 계산**: 연봉에서 비과세 소득 및 소득 공제를 차감한 금액이 과세표준이 됩니다. 예를 들어, 국민연금, 건강보험료 등의 공제 항목이나 인적 공제, 기타 공제항목 등을 차감해야 하지만, 여기서는 이러한 구체적인 공제를 적용하지 않은 상태로 설명드립니다.\n\n2. **소득세율**: 한국의 소득세는 다단계 누진세율로 이루어져 있으며, 1억 원에 해당하는 소득에 대한 적용 세율을 보면 다음과 같습니다.\n   - 1,200만 원 이하: 6%\n   - 1,200만 원 초과 ~ 4,600만 원 이하: 15%\n   - 4,600만 원 초과 ~ 8,800만 원 이하: 24%\n   - 8,800만 원 초과 ~ 1억 5천만 원 이하: 35%\n\n3. **계산**:\n   - 1,200만 원 × 6% = 72만 원\n   - (4,600만 원 - 1,200만 원) × 15% = 510만 원\n   - (8,800만 원 - 4,600만 원) × 24% = 1,008만 원\n   - (1억 원 - 8,800만 원) × 35% = 420만 원\n\n이를 모두 더하면, 총 소득세는 2,010만 원이 됩니다.\n\n단, 실제 소득세 납부액은 종합소득공제나 기타 세액공제를 반영하여 달라질 수 있습니다. 따라서, 신청 가능한 세금 혜택이나 공제 항목을 고려하지 않은 단순 계산임을 참고하시기 바랍니다. 정확한 세액 계산을 위해서는 세무 전문가와 상담하거나 국세청 등의 세금 계산기를 이용하는 것이 좋습니다.'